# 8.2

---

## 8.2.1.1

In [ ]:
import requests as rq   # 웹페이지 요청 라이브러리

url = 'https://quotes.toscrape.com/'
quote = rq.get(url)

quote.content[:1000]

In [ ]:
from bs4 import BeautifulSoup   # html 형식으로 뽑을 수 있게 잘 바꿔주는 라이브러리

quote_html = BeautifulSoup(quote.content, 'html.parser')
quote_html.head()

In [ ]:
# 명언이 존재하는 div 태그를 먼저 뽑음

quote_div = quote_html.find_all('div', class_='quote')

quote_div[0]

In [ ]:
# 명언이 존재하는 div 태그 내의 span 태그를 따로 뽑기

quote_span = quote_div[0].find_all('span', class_='text')

quote_span

In [ ]:
# span 태그 내의 텍스트 데이터만 추출

quote_span[0].text

In [ ]:
# 위의 과정을 한번에 처리하기

quote_div = quote_html.find_all('div', class_='quote')

[i.find_all('span', class_='text')[0].text for i in quote_div]

---

## 8.2.1.2

In [ ]:
# select 함수 내에 찾고자 하는 태그를 입력, 클래스명이 존재할 경우 . 이후에 클래스명 입력
# 여러 태그를 찾아 내려갈 때는 > 사용
quote_text = quote_html.select('div.quote > span.text')

quote_text

In [ ]:
quote_text_list = [i.text for i in quote_text]

quote_text_list

In [ ]:
# 말한 사람 이름 추출하기
quote_author = quote_html.select('div.quote > span > small.author')

quote_author_list = [i.text for i in quote_author]
quote_author_list

In [ ]:
# 말한 사람 정보 추출하기
quote_link = quote_html.select('div.quote > span > a')

quote_link

In [ ]:
# quote_link[0]['href'] - 속성 값 정보 추출

['https://quotes.toscrape.com' + i['href'] for i in quote_link]

---

## 8.2.1.3

In [28]:
# 웹사이트의 모든 페이지를 크롤링하는 예제

import requests as rq
from bs4 import BeautifulSoup
import time

text_list = []
author_list = []
infor_list = []

for i in range(1, 100):
    url = f'https://quotes.toscrape.com/page/{i}/'
    quote = rq.get(url)
    quote_html = BeautifulSoup(quote.content, 'html.parser')
    
    quote_text = quote_html.select('div.quote > span.text')
    quote_text_list = [i.text for i in quote_text]
    
    quote_author = quote_html.select('div.quote > span > small.author')
    quote_author_list = [i.text for i in quote_author]
    
    quote_link = quote_html.select('div.quote > span > a')
    quote_link_list = ['https://quotes.toscrape.com' + i['href'] for i in quote_link]
    
    if len(quote_text_list) > 0:
        text_list.extend(quote_text_list)
        author_list.extend(quote_author_list)
        infor_list.extend(quote_link_list)
        time.sleep(1)
        
    else:
        break

In [ ]:
import pandas as pd

quote_df = pd.DataFrame({'text': text_list, 'author': author_list, 'infor': infor_list})
quote_df

---

## 8.2.2


In [ ]:
# 기사 제목 뽑기 예제
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/news/news_list.nhn?mode=LSS2D&section_id=101&section_id2=258'
data = rq.get(url)
html = BeautifulSoup(data.content, 'html.parser')
html_select = html.select('dl > dd.articleSubject > a')

html_select[0:3]

In [ ]:
# title 속성에 있는 내용 뽑아내기
[i['title'] for i in html_select]

---

## 8.2.3

In [ ]:
# 크롤링을 원하는 데이터가 표 형태로 되어있을 경우

import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_countries_by_stock_market_capitalization'
tbl = pd.read_html(url)

tbl[0].head()

---

## 8.2.4

In [ ]:
# 한국거래소 상장공시시스템에서 POST 요청을 보내 데이터 뽑아내기

import requests as rq
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://kind.krx.co.kr/disclosure/todaydisclosure.do'
payload = {
   'method': 'searchTodayDisclosureSub',
   'currentPageSize': '15',
   'pageIndex': '1',
   'orderMode': '0',
   'orderStat':'D',
   'forward': 'todaydisclosure_sub',
   'chose': 'S',
   'todayFlag': 'N',
   'selDate': '2025-01-03'
}

data = rq.post(url, data=payload)       # post request
html = BeautifulSoup(data.content, 'html.parser')

# print(html)

In [ ]:
html_unicode = html.prettify()  # BeautifulSoup에서 파싱한 파서 트리를 유니코드 형태로 돌려놓기
html_unicode

In [ ]:
tbl = pd.read_html(html.prettify()) # html 문자열로 변환 후 read_html 사용
tbl[0].head()

---